In [ ]:
# This is the chatbot

In [ ]:
from dotenv import load_dotenv
import websockets
import json
import asyncio
import sqlite3
import requests

load_dotenv()

import os
oopenai_api_key = os.environ.get("OPENAI_API_KEY")
BOT_TOKEN = "5a3556bb2de44a73ab2e5643cb633a6c"
THREAD_ID = "default"

In [ ]:
def get_gpt3_response(prompt, stop=None):
    headers = {"Authorization": f"Bearer {oopenai_api_key}", "Content-Type": "application/json"}
    data = {"prompt": prompt, "max_tokens": 500, "temperature": 0, "model": "text-davinci-002"}
    if stop:
        data["stop"] = stop
    response = requests.post("https://api.openai.com/v1/completions", headers=headers, json=data)
    return response.json()["choices"][0]["text"]

talking_prompt = "This is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly. Sometimes the AI will say a crude joke in the form of a pun."

async def chatbot_messages():
    uri = f"ws://localhost:8000/ws/chat?thread_id={THREAD_ID}"
    messages = []
    async with websockets.connect(uri, extra_headers={"Cookie": f"Authorization=Bearer {BOT_TOKEN}"}) as ws:
        while True:
            try:
                raw_data = await asyncio.wait_for(ws.recv(), timeout=0.5)
                data = json.loads(raw_data)
                if data.get("replay", False):
                    messages.append(data)
                    continue
            except:
                break
    return messages

# Prompt chain example. 
# 1. Check if the "intent" of the message is to "run a query"

check_if_run_query_intent = "This is a chat between a human and an AI.\nAt this stage of the conversation, should the AI return a friendly response or should it execute SQL?"
getsql_prompt = "This is a chat where the human would like to run some SQL on SQLite, for a specific database (fivethirtyeight). What is the query that should be executed?\n\n"
conn = sqlite3.connect('../datasets/fivethirtyeight.db') 
c = conn.cursor()
c.execute("select * from sqlite_master limit 10")
res = c.fetchall()
getsql_prompt += "\n".join([str(x[1:]) for x in res])+ "\n\n"

friendly_response_prompt = "This is a chat between a human and a very friendly (maybe too friendly) AI system... what should the AI say?"

def get_chat_in_text(messages):
    return "\n".join([m["sender"]+":"+m["message"] for m in messages])

def prompt_chain_executor(messages):
    # given a list of messages, execute some prompt-chain and return a response
    prompt = check_if_run_query_intent + "\n====\n"+get_chat_in_text(messages[-1:])+"\n====\n\nShould the AI execute SQL to answer the last prompt?(y/n):"
    response = get_gpt3_response(prompt)
    if response.lstrip()[0] == "y":
        prompt = getsql_prompt +"\n====\n"+get_chat_in_text(messages[-1:])+"\n====\n\nWhat is the SQL query:"
        print(f"prompt is: {prompt}")
        response = get_gpt3_response(prompt)
        print(response)
        try:
            conn = sqlite3.connect('../datasets/fivethirtyeight.db') 
            c = conn.cursor()
            c.execute(response)
            res = c.fetchall()
        except Exception as e:
            res = f"Error executing SQL {e}"
        return f"I ran the query ```{response}``` and got a result {str(res)}"
    else:
        prompt = friendly_response_prompt + "\n====\n"+get_chat_in_text(messages[-1:])+"\n====\n\nWhat should the AI say:"
        response = get_gpt3_response(prompt)
        return response

async def chatbot():
    uri = f"ws://localhost:8000/ws/chat?thread_id={THREAD_ID}"
    messages = []
    async with websockets.connect(uri, extra_headers={"Cookie": f"Authorization=Bearer {BOT_TOKEN}"}) as ws:
        while True:
            try:
                raw_data = await asyncio.wait_for(ws.recv(), timeout=0.5)
                data = json.loads(raw_data)
                print("log:", data['message'])
                if data.get("replay", False):
                    messages.append(data)
                    continue
                if data["message"] == "break":
                    break
                if data["sender"] == "datAI":
                    # personal messages
                    continue
                if data.get("meta", False):
                    continue
                messages.append(data)
            except asyncio.TimeoutError:
                # not sure why I'm doing this, it felt important to have an "infinite" loop...
                continue

            # PROMPT CHAIN STUFF
            # Valid message to parse and likely respond to?
            # prompt_to_send = talking_prompt + "\n" + "\n".join([m["sender"]+":"+m["message"] for m in messages])+"\n"+"datAI:"
            # response = get_gpt3_response(prompt_to_send)
            response = prompt_chain_executor(messages)
            await ws.send(json.dumps({"message": response}))


In [ ]:
# messages = await chatbot_messages()
# prompt_chain_executor(messages)

In [ ]:
await chatbot()